# Automatic Speech Recognition with CTC Decoder

**Name**:

The first lab introduced the building blocks of an ASR system, including feature extraction and classification with an acoustic model (wav2vec2), which produced an *emission* matrix (probability for each character at each time step). From this emission matrix, we could compute the most likely character at each time step using a naïve *greedy* decoder. The drawback of such an approach is the lack of context, which can produce sequences of characters that do not correspond to actual words, and/or sequences of words that are incorrect / do not correspond to any language rules.

In this lab, we introduce the usage of a more advanced decoding technique that is based on [connectionist temporal classification](https://towardsdatascience.com/intuitively-understanding-connectionist-temporal-classification-3797e43a86c) (CTC). The general idea of such a decoder is to consider some context (sequences of characters, possible words, and possible sequences of words), in oder to yield more likely / realistic outputs than those given by the greedy decoder.

<center><a href="https://gab41.lab41.org/speech-recognition-you-down-with-ctc-8d3b558943f0">
    <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*XbIp4pn38rxL_DwzSeYVxQ.png" width="400"></a></center>

To do so, the CTC decoder relies on three main components:
    
- A **beam search**, which is an algorithm to efficently find the *best path* from the emission matrix, that is, the sequence of characters with highest probability (rather than the sequence of individually most likely characters).
- A **lexicon**, which is a mapping between token sequences (list of characters) and words. It is used to restrict the search space of the decoder to words that only belong to this dictionary (e.g., the word "azfpojazflj" does not exist in the English vocabulary).
- A **language model**, which specifies sequences of words that are more likely to occur than others. A common choice of language model is an $n$-gram, which is a statistical model for the probability of occurrence of a given word based on the previous $n$ ones (for instance, the sequence "the sky is" is more likely to be followed with "blue" rather than "trumpet").

The CTC decoder combines these ingredients to compute the score of several word sequences (or *hypothesis*), in order to find the best possible transcript. In this lab, we study the influence of the lexicon, language model, and the beam search size onto ASR performance from a practical perspective, without going into the technical details of the [beam search algorithm](https://www.width.ai/post/what-is-beam-search) or the [CTC loss](https://distill.pub/2017/ctc/), which can also be used for training the network (as we will see in lab 3).

**Note**: This lab is based on this [tutorial](https://pytorch.org/audio/main/tutorials/asr_inference_with_ctc_decoder_tutorial.html), which you can check for more details on CTC decoder parameters in torchaudio.

In [ ]:
import torch
import torchaudio
from torchaudio.models.decoder import ctc_decoder, download_pretrained_files
import IPython
import os
import fnmatch
import matplotlib.pyplot as plt
import time
torch.random.manual_seed(0)
MAX_FILES = 2 # lower this number for processing a subset of the dataset

In [ ]:
# Main dataset path - If needed, you can change it HERE but NOWHERE ELSE in the notebook!
data_dir = "../dataset/"

In [ ]:
# Speech and transcripts sub-directories paths
data_speech_dir = os.path.join(data_dir, 'speech')
data_transc_dir = os.path.join(data_dir, 'transcription')

## Preparation

As in the previous lab, we first load an example speech signal, and we display it. We also provide the function to get the true transcript and compute the WER. Finally, we load the wav2vec2 acoustic model.

In [ ]:
# Example file
audio_file = '61-70968-0001.wav'
audio_file_path = os.path.join(data_speech_dir, audio_file)
print(f"Audio file path: {audio_file_path}")

waveform, sr = torchaudio.load(audio_file_path, channels_first=True)
IPython.display.Audio(data=waveform, rate=sr)

In [ ]:
# We provide the function for loading the true transcript and computing the WER
def get_true_transcript(transc_file_path):
    with open(transc_file_path, "r") as f:
        true_transcript = f.read()
    true_transcript = true_transcript.lower().split()
    return true_transcript

def get_wer(true_transcript, est_transcript):
    wer = torchaudio.functional.edit_distance(true_transcript, est_transcript) / len(true_transcript)
    return wer
    

In [ ]:
# Load and display the true transcription
transc_file_path = os.path.join(data_transc_dir, audio_file.replace('wav', 'txt'))
true_transcript = get_true_transcript(transc_file_path)
print(true_transcript)

In [ ]:
# Load the acoustic model
model_name = 'WAV2VEC2_ASR_BASE_100H'
bundle = getattr(torchaudio.pipelines, model_name)
acoustic_model = bundle.get_model()
labels = bundle.get_labels()

# Apply the model to the waveform to get the emission tensor
with torch.inference_mode():
    emission, _ = acoustic_model(waveform)
    num_time_steps = emission.shape[1]

## CTC Decoder

The CTC decoder can be constructed directly by using the `ctc_decoder` function in torchaudio. In addition to the parameters related to the beam search (we will study them later on), it takes as inputs:
- the list of tokens, in order to map emissions to characters in the classifier.
- the path to the lexicon, expected as a .txt file containing, on each line, a word followed by its space-split tokens (and special end-of-sequence token `|`):

```
# lexicon.txt
a     a |
able  a b l e |
about a b o u t |
...
```
- the path to the language model, expected as a .bin file.

All these are assembled in pretrained files that can be downloaded using the `download_pretrained_files` function (this might take some time as the language model can be large), and then used to contruct the decoder.

In [ ]:
# Download the files corresponding to (pretrained) language model, which comes with lexicon and tokens
files = download_pretrained_files("librispeech")
path_lm_tokens = files.tokens
path_lm_lexicon = files.lexicon
path_lm = files.lm

print(path_lm_tokens)
print(path_lm_lexicon)
print(path_lm)

In [ ]:
# Vizualize the first 10 tokens (includes the blank and end-of-word token)
with open(path_lm_tokens, 'r') as f:
    tok = f.read().splitlines()
print("\n".join(tok[:10]))

In [ ]:
# Vizualize the lexicon content (first 10 entries)
with open(path_lm_lexicon, 'r') as f:
    lex = f.read().splitlines()
print("\n".join(lex[:10]))

In [ ]:
# To obtain the line(s) corresponding to a word in the lexicon, you can use the following:
w = 'hello'
lex_w = [line for line in open(path_lm_lexicon) if line.startswith(w + '\t')] # it's a list since there could be different pronunciation for the same word
print(lex_w)

In [ ]:
# Instanciate the CTC decoder
decoder = ctc_decoder(
    lexicon=path_lm_lexicon,
    tokens=path_lm_tokens,
    lm=path_lm,
)

# Apply the decoder to the `emission` tensor, and get the first element (batch_size=1) and best hypothesis
ctc_decoder_result = decoder(emission)[0][0]

The decoder output `ctc_decoder_result` contains many fields, including the predicted token IDs, and a `.words` field that contains the transcript as a list of strings.

In [ ]:
# Get the token IDs using the .tokens field
ctc_decoder_indices = ctc_decoder_result.tokens
print(f"Token indices: {ctc_decoder_indices}")

# Display the transript using the .words field
print(f"Words: {ctc_decoder_result.words}")

In [ ]:
# Get the token IDs using the .tokens field
ctc_decoder_indices = ctc_decoder_result.tokens
print(f"Token indices: {ctc_decoder_indices}")

# Get the words vis the .words field
print(f"Words: {ctc_decoder_result.words}")

### Greedy decoder

The greedy decoder we have seen in the first lab is a particular case of the CTC decoder when no langage model / lexicon is provided. It can be constructed by simply passing `None` as corresponding input arguments in the `ctc_decoder` function.

In [ ]:
# TO DO: Construct a greedy decoder (no LM / lexicon), and apply it to the emission matrix
decoder_nolm = ctc_decoder(
    lexicon=None,
    tokens=path_lm_tokens,
    lm=None
)

ctc_nolm_result = decoder_nolm(emission)[0][0]

In [ ]:
# Since no language model is provided, the .words field returns an empty list
print(ctc_nolm_result.words)

# Then we have to manually convert token IDs to tokens using the decoder.idxs_to_tokens method
# (+ a bit of postprocessing)
ctc_nolm_tokens = decoder.idxs_to_tokens(ctc_nolm_result.tokens)
ctc_nolm_transcript = "".join(ctc_nolm_tokens).replace("|", " ").strip().split() 
ctc_nolm_transcript = [w.lower() for w in ctc_nolm_transcript]

print(f"No LM Transcript: {ctc_nolm_transcript}")

## Influence of the language model

The language model is also expected to have a strong impact onto performance, since it guides the decoder towards more likely word sequences.

<span style="color:red"> **Exercise 1**</span>. Compare the CTC decoder using `librispeech-4-gram` files (lexicon, token, and language model downloaded above in this script) and the greedy decoder. To that end, for each decoder, perform ASR on the whole dataset (feel free to reuse/adapt code from the previous lab) and compute the mean WER. Can you interprete the results?

In [ ]:
def find_files(directory, pattern='*.wav'):
    """Recursively finds all files matching the pattern."""
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(filename)
    files = sorted(files)
    return files


def process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=True, max_files=None):

    # Get the list of files in the dataset folder
    audio_files = find_files(data_speech_dir)

    # Take a subset of files
    nfiles = len(audio_files)
    if max_files:
        nfiles = min(nfiles, max_files)
    audio_files = audio_files[:nfiles]

    # Initialize lists containing true and estimated transcripts, as well as WER
    true_transcript_all = []
    est_transcript_all = []
    wer_all = []

    for iaf, audio_file in enumerate(audio_files):
        
        # Get files path
        audio_file_path = os.path.join(data_speech_dir, audio_file)
        transc_file_path = os.path.join(data_transc_dir, audio_file.replace('wav', 'txt'))

        # Load an audio signal
        waveform, sr = torchaudio.load(audio_file_path, channels_first=True)

        # Apply acoustic model and decoder
        with torch.inference_mode():
            emission, _ = acoustic_model(waveform)
            ctc_decoder_results = decoder(emission)[0][0]

        ctc_tokens = decoder.idxs_to_tokens(ctc_decoder_results.tokens)
        est_transcript = "".join(ctc_tokens).replace("|", " ").strip().split()
        est_transcript = [w.lower() for w in est_transcript]

        # Load the true transcription
        true_transcript = get_true_transcript(transc_file_path)
        
        # Compute WER
        wer = get_wer(true_transcript, est_transcript)
        wer_all.append(wer)

        est_transcript_all.append(est_transcript)
        true_transcript_all.append(true_transcript)
        
        # Display results
        if verbose:
            print(f"File {iaf+1} / {nfiles}")
            print('Estimated transcript: ', est_transcript)
            print('True transcript: ', true_transcript)
            print(f"WER: {wer*100:.1f} %")

    wer_mean = torch.FloatTensor(wer_all).mean()

    return wer_mean, est_transcript_all, true_transcript_all

In [ ]:
# CTC decoder with the 4-gram language model
decoder = ctc_decoder(
    lexicon=path_lm_lexicon,
    tokens=path_lm_tokens,
    lm=path_lm,
)
wer_mean = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)[0]
print(f"LM: --- WER: {wer_mean*100:.1f} %")

# Greedy decoder
decoder = ctc_decoder(
    lexicon=None,
    tokens=labels,
    lm=None,
)
wer_mean = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)[0]
print(f"Greedy Decoder --- WER: {wer_mean*100:.1f} %")

## Influence of the lexicon

The lexicon is expected to have a strong influence on ASR performance, since it constrains the decoder to produce only words that belong to this lexicon, therefore avoiding to procude words that potentially do not exist in a language or given corpus. Here, we propose to use a custom lexicon that only contains words that are in the dataset.

<span style="color:red"> **Exercise 2**</span>. Perform ASR using such a custom lexicon. To that end:
- Build a vocabulary (list of words) from the transcript files in the dataset (load the transcripts and remove duplicates).
- Filter the downloaded lexicon by only keeping words from the vocabulary. Write this custom lexicon as a `.txt` file.
- Create a dedocer that uses the 4-gram language model and this custom lexicon. Perform ASR on the dataset / compute the WER. What do you observe?

In [ ]:
# Fonction to get a flat list from a list of list
def flatten_list(list_of_lists):
    return [x for L in list_of_lists for x in L]

# Read all the transcripts in the dataset
transcr_files = find_files(data_transc_dir, pattern='*.txt')
true_transcript = []
for f in transcr_files:
    transc_file_path = os.path.join(data_transc_dir, f)
    true_transcript.append(get_true_transcript(transc_file_path))

# Flatten the list and remove duplicates
vocab_dataset = list(set(flatten_list(true_transcript)))
print('Words in the dataset:', len(vocab_dataset))

# Filter the provided lexicon by keeping only words in our dataset
custom_lexicon = []
for w in vocab_dataset:
    wl = [line for line in open(files.lexicon) if line.startswith(w + '\t')]
    custom_lexicon.append(wl)

# again, flatten it
custom_lexicon = flatten_list(custom_lexicon)

# There are less entries in the lexicon than in our list of words from the dataset: several words are indeed not registered in the lexicon (we could add them manually)
print('Entries in the custom lexicon: ', len(custom_lexicon))

# Record this lexicon
with open("mylexicon.txt", "w") as f:
    f.writelines(custom_lexicon)

In [ ]:
# CTC decoder with the language model + a custom lexicon
decoder = ctc_decoder(
    lexicon="mylexicon.txt",
    tokens=path_lm_tokens,
    lm=path_lm,
)
wer_mean = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)[0]
print(f"Custom lexicon --- WER: {wer_mean*100:.1f} %")

**Comment**: This custom lexicon produces a better transcript (lower WER) than the more general lexicon. This makes sense since our custom lexicon is specifically tailored for this dataset, and then words outside the dataset cannot be predicted. Nonetheless there are still some errors: besides the acoustic model, these can be due to several words in the dataset not being actually in the original downloaded lexicon, therefore the model cannot predict them using solely the language model.

## Beam search parameters

The beam search algorithm used in the CTC decoder depends on other parameters, such as `nbest` which determines the number of hypotheses to return, or `lm_weight` which adjust the relative importance of the language model vs. the acoustic model predictions. Here we only focus on `beam_size`, which determines the maximum number of best hypotheses to hold after each decoding step. Using larger beam sizes allows for exploring a larger range of possible hypotheses which can produce hypotheses with higher scores, which really is [the core](https://distill.pub/2017/ctc/) of the beam search algorithm.

<span style="color:red"> **Exercise 3**</span>. Perform ASR on the whole dataset folder for several values of the beam search size parameter: `beam_size` $\in [1, 10, 100]$ (use the original downloaded lexicon, not the custom one). Compute the WER and the computation time (e.g., via the [time](https://docs.python.org/3/library/time.html#time.time) package). What do you observe?

In [ ]:
beam_sizes = [1, 10, 100]

for beam_size in beam_sizes:
    decoder = ctc_decoder(
        lexicon=path_lm_lexicon,
        tokens=path_lm_tokens,
        lm=path_lm,
        beam_size=beam_size,
    )

    time_start = time.time()
    wer_mean = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)[0]
    time_ellapsed = time.time() - time_start

    print(f"Beam search size: {beam_size} --- WER: {wer_mean*100:.1f} % --- Time: {time_ellapsed:.1f}")

**Comment**: Using a beam search size that is too large (i.e., building too many hypothesis when decoding) does not improve performance compared to a lower value, but increases the computational cost. Thus, it is recommended to adjust this value in order to find a tradeoff between accuracy and speed.

## Segmentation / alignment

**Unfinished / non working : can be changed to add an extra exercice if needed**

In [ ]:
print(ctc_decoder_result.timesteps)

In [ ]:
word = {}
word['start'] = 0
word['end'] = 10
word['label'] = 'l'
word_segments = []
word_segments.append(word)

In [ ]:
# TODO: faire coder le time end (en utilisant '|' ) 

In [ ]:
def plot_alignments_wav(waveform, emission, tokens, timesteps, sample_rate):
    waveform /= waveform.max()
    t = torch.arange(waveform.size(0)) / sample_rate
    ratio = waveform.size(0) / emission.size(1) / sample_rate

    chars = []
    words = []
    word_start = None
    for token, timestep in zip(tokens, timesteps * ratio):
        if token == "|":
            if word_start is not None:
                words.append((word_start, timestep))
            word_start = None
        else:
            chars.append((token, timestep))
            if word_start is None:
                word_start = timestep

    fig = plt.figure(figsize=(12,4))
    plt.plot(t, waveform)
    for token, timestep in chars:
        plt.annotate(token, (timestep, 1))
    for word_start, word_end in words:
        plt.axvspan(word_start, word_end, alpha=0.1, color="red")
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.tight_layout()
    plt.show()


ctc_nolm_tokens = decoder.idxs_to_tokens(ctc_nolm_result.tokens)
plot_alignments_wav(waveform[0], emission, ctc_nolm_tokens, ctc_decoder_result.timesteps, sr)

In [ ]:
def plot_alignments(segments, word_segments, waveform, sample_rate=bundle.sample_rate):

    # The original waveform
    ratio = waveform.size(0) / sample_rate / trellis.size(0)
    ax2.specgram(waveform, Fs=sample_rate)
    for word in word_segments:
        x0 = ratio * word.start
        x1 = ratio * word.end
        ax2.axvspan(x0, x1, facecolor="none", edgecolor="white", hatch="/")
        ax2.annotate(f"{word.score:.2f}", (x0, sample_rate * 0.51), annotation_clip=False)

    for seg in segments:
        if seg.label != "|":
            ax2.annotate(seg.label, (seg.start * ratio, sample_rate * 0.55), annotation_clip=False)
    ax2.set_xlabel("time [second]")
    ax2.set_yticks([])
    fig.tight_layout()


plot_alignments(
    segments,
    word_segments,
    waveform[0],
)

In [ ]:
def display_segment(waveform, word, num_time_steps, sr=16000):
    ratio = waveform.shape[-1] / num_time_steps
    x0 = int(ratio * word['start'])
    x1 = int(ratio * word['end'])
    print(f"Character: {word['label']}")
    segment = waveform[:, x0:x1]
    return IPython.display.Audio(segment, rate=sr)

In [ ]:
word = word_segments[0]
display_segment(waveform, word, num_time_steps, sr=16000)